# Name: Nalet Meinen¶

# Assignment 2 ATML 2020
## Classification with limited data
ImageNet is a well known dataset with 1000 image classes. We will be working on a subset of the dataset (60k images, 100 classes, 600 images per class 80$\times$80 pixels, RGB) and train a model to classify an image into one of the 100 classes. The dataset is located under the "data" directory. Training and validation data splits are under "data/train" and "data/val" directories respectively. Both splits consist of 100 directories, each representing an object category.

## Task 1. Implement ImageNetLimited class for data loading in datasets.py file

In [1]:
import numpy as np
import torch
from torchvision.datasets import ImageFolder
from torchvision.transforms import Resize, ToTensor, Normalize, Compose, CenterCrop
from datasets import ImageNetLimited
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    print("This nootebook ran on",device.type,"With number of GPU:",torch.cuda.device_count())

train_dir = 'data/train'
validation_dir = 'data/val'
len_classes = 100

# write your code
transforms = Compose(   [Resize(64),
                        ToTensor(),
                        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

_train_dataset = ImageFolder(train_dir, transform=transforms)
_val_dataset = ImageFolder(validation_dir, transform=transforms)

train_dataset = ImageNetLimited(_train_dataset)
val_dataset = ImageNetLimited(_val_dataset)

batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0) #set num_workers = 0, to fix BrokenPipeError in Windows
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

This nootebook ran on cuda With number of GPU: 1


## Task 2. CNN Architecture
Design and implement a Convolutional Neural Network architecture for image classification in a **ConvNet** class in the notebook. Some examples of popular classification models are: AlexNet, VGG, ResNet, ... Justify your design choices in the report. The input to your model must be an image of size $64 \times 64$ pixels.

In [2]:
import torch.nn as nn

class ConvNet(nn.Module):
    def __init__(self, num_classes=100):
        super(ConvNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

## Task 3. Train Model
Implement training and evaluation code for your model. Choose an appropriate loss function and evaluate the model on the validation set using classification accuracy. You are not allowed to use a pre-trained model (must train from scratch on the provided data).<br>
<font color='red'>Your model should achieve an accuracy of at least 40.0% on the validation set (Model with performance smaller than 40.0% will result in 0 points for this task).</font><br>

In [3]:
from utils_train import fit
model_conv = ConvNet()
model_conv = model_conv.to(device)
learning_rate = 0.0001
optimizer = torch.optim.Adam(model_conv.parameters(), lr=learning_rate)
n_epochs = 30
loss_fn = nn.CrossEntropyLoss()
curves_conv2 = fit(train_dataloader, val_dataloader, model_conv, optimizer, loss_fn, n_epochs)


Test average loss: 4.1295, accuracy: 4.314
Epoch 1/30: train_loss: 4.3290, train_accuracy: 2.9200, val_loss: 4.1295, val_accuracy: 4.3142



Test average loss: 3.9232, accuracy: 7.262
Epoch 2/30: train_loss: 4.0113, train_accuracy: 6.0517, val_loss: 3.9232, val_accuracy: 7.2618



Test average loss: 3.7800, accuracy: 9.606
Epoch 3/30: train_loss: 3.8165, train_accuracy: 8.8233, val_loss: 3.7800, val_accuracy: 9.6064



Test average loss: 3.7615, accuracy: 10.518
Epoch 4/30: train_loss: 3.6637, train_accuracy: 11.1383, val_loss: 3.7615, val_accuracy: 10.5175



Test average loss: 3.6559, accuracy: 11.693
Epoch 5/30: train_loss: 3.5341, train_accuracy: 13.1217, val_loss: 3.6559, val_accuracy: 11.6932


...


Test average loss: 8.1216, accuracy: 13.405
Epoch 26/30: train_loss: 0.6241, train_accuracy: 79.7433, val_loss: 8.1216, val_accuracy: 13.4048



Test average loss: 8.4560, accuracy: 13.442
Epoch 27/30: train_loss: 0.5799, train_accuracy: 81.1700, val_loss: 8.4560, val_accuracy: 13.4416



Test average loss: 8.5734, accuracy: 13.733
Epoch 28/30: train_loss: 0.5454, train_accuracy: 82.4117, val_loss: 8.5734, val_accuracy: 13.7330



Test average loss: 8.6479, accuracy: 13.435
Epoch 29/30: train_loss: 0.5168, train_accuracy: 83.2700, val_loss: 8.6479, val_accuracy: 13.4349



Test average loss: 8.6408, accuracy: 13.070
Epoch 30/30: train_loss: 0.4835, train_accuracy: 84.3617, val_loss: 8.6408, val_accuracy: 13.0698



## Task 4. Ablations
Try to find the best performing model by tuning the model design and hyper-parameters on the validation set. Perform ablation experiments to illustrate the effect of the most important hyper-parameters. Some examples of ablations: training parameters (e.g., optimizer, learning rates, batch size), network architecture (e.g., number of layers, number of units, activation function, normalization layers), model regularization (e.g., data augmentation, dropout, weight decay, early stopping), test-time augmentation, etc...  <br>**Perform at least 5 ablations and report the performance of each on the validation set.**

In [4]:
# write your code

## Task 5. Model Errors
Evaluate the trained model on the validation set and plot 10 random mistakes that your model made.

In [5]:
# write your code

## Task 6. Competition time!
Read the images from "data/test" folder. There are no labels for these images. Run your best model on these images and save the image IDs (names) and predicted label in a file LastName.csv. You will receive a link via email to upload the CSV file to  an online system which will give you the score of your model on the held-out test set. Top 5 students with at least 40% classification accuracy will obtain bonus points.

In [6]:
# write your code